# Grouping and aggregation with pandas

## Aggregation and reduction

Similar to NumPy, pandas supports data aggregation and reduction functions 
such as computing sums or averages. By _"aggregation"_ or _"reduction"_ 
we mean that the result of a computation has a lower dimension than the original data: for example, the mean reduces a series of observations (1 dimension) into a scalar value (0 dimensions).

Unlike NumPy, these operations
can be applied to subsets of the data which have been
grouped according to some criterion. 

Such operations are often referred to as *split-apply-combine* (see the official [user guide](https://pandas.pydata.org/docs/user_guide/groupby.html)) as they involve these three steps:

1. *Split* data into groups based on some criteria;
2. *Apply* some function to each group separately; and
3. *Combine* the results into a single `DataFrame` or `Series`.

See also the pandas [cheat sheet](https://pandas.pydata.org/Pandas_Cheat_Sheet.pdf) for an illustration of such operations.

We first set the path pointing to the folder which contains the data files used in this lecture. You may need to adapt it to your own environment.

In [1]:
# Uncomment this to use files in the local data/ directory
DATA_PATH = '../../data'

# Uncomment this to load data directly from GitHub
# DATA_PATH = 'https://raw.githubusercontent.com/richardfoltyn/TECH2-H25/main/data'

### Aggregations of whole Series or DataFrames

The simplest way to perform data reduction is to invoke the desired
function on the entire `DataFrame`.
We illustrate this using the Titanic dataset which we have encountered 
in the previous lectures.

We first load the data and tabulate the columns present in the `DataFrame`:

In [2]:
import pandas as pd

# Path to Titanic passenger data CSV file
file = f'{DATA_PATH}/titanic.csv'

# Read in Titanic passenger data, set PassenderId column as index
df = pd.read_csv(f'{DATA_PATH}/titanic.csv', index_col='PassengerId')

# Tabulate columns and number of observations
df.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Index: 891 entries, 1 to 891
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Name      891 non-null    object 
 3   Sex       891 non-null    object 
 4   Age       714 non-null    float64
 5   Ticket    891 non-null    object 
 6   Fare      891 non-null    float64
 7   Cabin     204 non-null    object 
 8   Embarked  889 non-null    object 
dtypes: float64(2), int64(2), object(5)
memory usage: 69.6+ KB


We can now apply the 
[`mean()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.mean.html) 
method to all numerical columns to compute the average for each column:

In [3]:
# Compute mean of all numerical columns|
df.mean(numeric_only=True)

Survived     0.383838
Pclass       2.308642
Age         29.699118
Fare        32.204208
dtype: float64

Methods such as [`mean()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.mean.html) 
are by default applied column-wise to each
column. The `numeric_only=True` argument is used to discard
all non-numeric columns (depending on the version of pandas, `mean()` will
issue a warning if there are non-numerical columns in the `DataFrame`).

One big advantage over NumPy is that missing values (represented
by `np.nan`) are automatically ignored, as the following code demonstrates:

In [5]:
import numpy as np

# mean() automatically drops missing observations
mean_pandas = df['Age'].mean()

# np.mean() returns NaN since some ages are missing (coded as NaN)
mean_numpy = np.mean(df['Age'].to_numpy())

print(f'Mean using Pandas: {mean_pandas:.1f}')
print(f'Mean using NumPy:  {mean_numpy:.1f}')

Mean using Pandas: 29.7
Mean using NumPy:  nan


For this reason, NumPy implements an additional set of aggregation functions which drop NaNs, for example [`np.nanmean()`](https://numpy.org/doc/2.0/reference/generated/numpy.nanmean.html).

### Aggregations of subsets of data (grouping)

Applying aggregation functions to the entire `DataFrame` is similar
to what we can do with NumPy. The added flexibility of pandas
becomes obvious once we want to apply these functions to subsets of
data, i.e., groups which we define based on values or index labels.

For example, we can easily group passengers by class using
[`groupby()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.groupby.html):

In [4]:
import pandas as pd

# Import Titanic data set, set PassenderId column as index
df = pd.read_csv(file, index_col='PassengerId')

# Group observations by accommodation class (first, second, third)
groups = df.groupby(['Pclass'])

Here `groups` is a special pandas objects which can subsequently be
used to process group-specific data. To compute the group-wise
averages, we can simply run

In [5]:
groups.mean(numeric_only=True)

,Survived,Age,Fare
Pclass,,,
1,0.629630,38.233441,84.154687
2,0.472826,29.877630,20.662183
3,0.242363,25.140620,13.675550


Groups support column indexing: if we want to only compute the
total fare paid by passengers in each class, we can do this as follows:

In [6]:
groups['Fare'].sum()

Pclass
1    18177.4125
2     3801.8417
3     6714.6951
Name: Fare, dtype: float64

#### Built-in aggregations

There are numerous routines to aggregate grouped data, for example:

- [`mean()`](https://pandas.pydata.org/docs/reference/api/pandas.core.groupby.DataFrameGroupBy.mean.html):
    compute average within each group
- [`sum()`](https://pandas.pydata.org/docs/reference/api/pandas.core.groupby.DataFrameGroupBy.sum.html):
    sum values within each group
- [`std()`](https://pandas.pydata.org/docs/reference/api/pandas.core.groupby.DataFrameGroupBy.std.html), 
    [`var()`](https://pandas.pydata.org/docs/reference/api/pandas.core.groupby.DataFrameGroupBy.var.html): 
    within-group standard deviation and variance
-   [`median()`](https://pandas.pydata.org/docs/reference/api/pandas.core.groupby.DataFrameGroupBy.median.html):
    compute median within each group    
- [`quantile()`](https://pandas.pydata.org/docs/reference/api/pandas.core.groupby.DataFrameGroupBy.quantile.html):
    compute quantiles within each group
- [`size()`](https://pandas.pydata.org/docs/reference/api/pandas.core.groupby.DataFrameGroupBy.size.html): 
    number of observations in each group
- [`count()`](https://pandas.pydata.org/docs/reference/api/pandas.core.groupby.DataFrameGroupBy.count.html):
    number of non-missing observations in each group
- [`first()`](https://pandas.pydata.org/docs/reference/api/pandas.core.groupby.DataFrameGroupBy.first.html), 
    [`last()`](https://pandas.pydata.org/docs/reference/api/pandas.core.groupby.DataFrameGroupBy.last.html): 
    first and last elements in each group
-   [`min()`](https://pandas.pydata.org/docs/reference/api/pandas.core.groupby.DataFrameGroupBy.min.html), 
    [`max()`](https://pandas.pydata.org/docs/reference/api/pandas.core.groupby.DataFrameGroupBy.max.html): 
    minimum and maximum elements within a group

See the [official documentation](https://pandas.pydata.org/docs/user_guide/groupby.html#built-in-aggregation-methods) for a complete list.

*Example: Number of elements within each group*

In [9]:
groups.size()       # return number of elements in each group

Pclass
1    216
2    184
3    491
dtype: int64

Note that `size()` and `count()` are two different functions. The former returns the group sizes (and the return value is a `Series`), whereas `count()` returns the number of non-missing observations for *each* column.

*Example: Return first observation of each group*

In [10]:
groups[['Survived', 'Age', 'Sex', 'Fare']].first()      # return first observation in each group

,Survived,Age,Sex,Fare
Pclass,,,,
1,1,38.0,female,71.2833
2,1,14.0,female,30.0708
3,0,22.0,male,7.2500


<div class="alert alert-info">
<h3> Your turn</h3>
Use the Titanic data set to perform the following aggregations:
<ol>
    <li>Compute the average survival rate by sex (stored in the <TT>Sex</TT> column).</li>
    <li>Count the number of passengers aged 50+. Compute the average survival rate by sex for this group.</li>
    <li>Count the number of passengers below the age of 20 by class and sex. Compute the average survival rate for this group by class and sex.</li>
</ol>
</div>

In [20]:
file2 = f'{DATA_PATH}/titanic.csv'
df1 = pd.read_csv(file2)

In [32]:
df20 = df1.loc[df1['Age']<20]
df20grouped = df20.groupby(['Pclass', 'Sex'])
df20grouped.size()

Pclass  Sex   
1       female    14
        male       7
2       female    16
        male      19
3       female    45
        male      63
dtype: int64

In [33]:
df20grouped['Survived'].mean()

Pclass  Sex   
1       female    0.928571
        male      0.571429
2       female    1.000000
        male      0.526316
3       female    0.533333
        male      0.190476
Name: Survived, dtype: float64

In [22]:
groups_survival = df1.groupby(['Sex'])
groups_survival['Survived'].mean()

Sex
female    0.742038
male      0.188908
Name: Survived, dtype: float64

In [25]:
df1

,PassengerId,Survived,Pclass,Name,Sex,Age,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss Laina",female,26.0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss Margaret Edith",female,19.0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss Catherine Helen ""Carrie""",female,NaN,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,111369,30.0000,C148,C


In [30]:
df50 = df1.loc[df1['Age']>50]

df50count = df1.shape[0]
print(df50count)


groups50_survival = df50.groupby(['Sex'])
groups50_survival['Survived'].mean()

891


Sex
female    0.941176
male      0.127660
Name: Survived, dtype: float64

#### Writing custom aggregations

We can create custom aggregation routines by calling 
[`agg()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.agg.html)
(short-hand for [`aggregate()`](https://pandas.pydata.org/docs/reference/api/pandas.core.groupby.DataFrameGroupBy.aggregate.html))
on the grouped object. These functions operate on one column at a time, so it is only possible to use observations from that column for computations. 

For example, we can alternatively call the built-in aggregation functions we just covered via `agg()`:

In [35]:
# Calculate group means in needlessly complicated way
# groups["Age"].agg("mean")

# More direct approach:
groups["Age"].mean()

Pclass
1    38.233441
2    29.877630
3    25.140620
Name: Age, dtype: float64

On the other hand, we _have to_ use `agg()` if there is no built-in function to perform the desired aggregation.
To illustrate, imagine that we want to count the number of passengers aged 40+ in each class. There is no built-in function to achieve this, so we need to use `agg()` combined with a custom function to perform the desired aggregation:

In [12]:
import numpy as np

# Apply a custom aggregation using a lambda expression
groups['Age'].agg(lambda x: np.sum(x >= 40))

Pclass
1    81
2    37
3    45
Name: Age, dtype: int64

In this example, we use a [`lambda` expression](https://www.w3schools.com/python/python_lambda.asp)
 to define the custom aggregation function in place. This is a shorthand notation which is equivalent to defining a custom function first:

In [13]:
# Define a custom aggregation function
def fcn(x):
    return np.sum(x >= 40)

# Apply the custom aggregation function to the 'Age' column
groups['Age'].agg(fcn)

Pclass
1    81
2    37
3    45
Name: Age, dtype: int64

Note that we called `agg()` only on the column `Age`, otherwise
the function would be applied to every column separately, which is not
what we want.

#### Applying multiple functions at once

It is possible to apply multiple functions in a single call by passing a list of functions. These can be passed as strings or as callables (functions).

*Example: Applying multiple functions to a **single** column*

 To compute the mean and median passenger age by class, we proceed as follows:

In [14]:
groups['Age'].agg(['mean', 'median'])

,mean,median
Pclass,,
1,38.233441,37.0
2,29.877630,29.0
3,25.140620,24.0



Note that we could have also specified these function by passing references to the corresponding NumPy functions instead:

In [15]:
groups['Age'].agg([np.mean, np.median])

C:\Users\sophi\AppData\Local\Temp\ipykernel_8636\3990418967.py:1: FutureWarning: The provided callable <function mean at 0x000001E74224D800> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  groups['Age'].agg([np.mean, np.median])
C:\Users\sophi\AppData\Local\Temp\ipykernel_8636\3990418967.py:1: FutureWarning: The provided callable <function median at 0x000001E742367EC0> is currently using SeriesGroupBy.median. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "median" instead.
  groups['Age'].agg([np.mean, np.median])


,mean,median
Pclass,,
1,38.233441,37.0
2,29.877630,29.0
3,25.140620,24.0


The following more advanced syntax allows us to create new column names using existing columns and some operation:

```python
    groups.agg(
        new_column_name1=('column_name1', 'operation1'),
        new_column_name2=('column_name2', 'operation2'),
        ...
    )
```
This is called ["named aggregation"](https://pandas.pydata.org/docs/user_guide/groupby.html#named-aggregation)
as the keywords determine the output column _names_.

*Example: Applying multiple functions to **multiple** columns*

The following code computes the average age and the highest fare in a single aggregation:

In [16]:
groups.agg(
    average_age=('Age', 'mean'), 
    max_fare=('Fare', 'max')
)

,average_age,max_fare
Pclass,,
1,38.233441,512.3292
2,29.877630,73.5000
3,25.140620,69.5500


Finally, the most flexible aggregation method is `apply()` which calls a
given function, passing the _entire_ group-specific subset of data (including
all columns) as an argument. You need to use apply if data from more than one column is required to compute a statistic of interest.

<div class="alert alert-info">
<h3> Your turn</h3>
Use the Titanic data set to perform the following aggregations:
<ol>
    <li>Compute the minimum, maximum and average age by embarkation port (stored in the column <TT>Embarked</TT>) in a single <TT>agg()</TT> operation.
    Note that there are several ways to solve this problem.</li>
    <li>Compute the number of passengers, the average age and the fraction of women by embarkation port in a single <TT>agg()</TT> operation. 
    <br/>
    <i>Hint:</i>To compute the fraction of women, you can either use a <TT>lambda</TT> expressions, or you first create a numerical indicator variable for females
    (as we did in the workshop).
    </li>
</ol>
</div>

## Transformations

In the previous section, we combined grouping and reduction, i.e., data at the group level was reduced to a single statistic such as the mean. Alternatively, we can combine grouping with the
[`transform()`](https://pandas.pydata.org/docs/reference/api/pandas.core.groupby.DataFrameGroupBy.transform.html) function which assigns the result of a computation to each observation within a group and consequently leaves the number of observations unchanged.

For example, for _each_ observation we could compute the average fare by class as follows:

In [17]:
df['Avg_Fare'] = df.groupby('Pclass')[['Fare']].transform('mean')

# Print results for each institution
df[['Pclass', 'Fare', 'Avg_Fare']].head(10)

,Pclass,Fare,Avg_Fare
PassengerId,,,
1,3,7.2500,13.675550
2,1,71.2833,84.154687
3,3,7.9250,13.675550
4,1,53.1000,84.154687
5,3,8.0500,13.675550
6,3,8.4583,13.675550
7,1,51.8625,84.154687
8,3,21.0750,13.675550
9,3,11.1333,13.675550


As you can see, instead of collapsing the `DataFrame` to only 3 observations (one for each class), the number of observations remains the same, and the average fare is constant within each class. 

When would we want to use `transform()` instead of aggregation? Such use cases arise whenever we want to perform computations that include the individual value as well as an aggregate statistic.

*Example: Deviation from average fare*

Assume that we want to compute how much each passenger's fare differed from the average fare in their respective class. We could compute this using `transform()` as follows:

In [18]:
import numpy as np

# Compute difference of passenger's fare and avg. fare paid within class
df['Fare_Diff'] = df.groupby('Pclass')['Fare'].transform(lambda x: x - np.mean(x))

# Print relevant columns
df[['Pclass', 'Fare', 'Fare_Diff']].head(10)

,Pclass,Fare,Fare_Diff
PassengerId,,,
1,3,7.2500,-6.425550
2,1,71.2833,-12.871387
3,3,7.9250,-5.750550
4,1,53.1000,-31.054687
5,3,8.0500,-5.625550
6,3,8.4583,-5.217250
7,1,51.8625,-32.292187
8,3,21.0750,7.399450
9,3,11.1333,-2.542250


<div class="alert alert-info">
<h3> Your turn</h3>
Use the Titanic data set to perform the following aggregations:
<ol>
    <li>Compute the <i>excess</i> fare paid by each passenger relative to the minimum fare by embarkation port and class, i.e., compute <i>Fare - min(Fare)</i>
        by port and class.</li>
</ol>
</div>

In [36]:
file3 = f'{DATA_PATH}/titanic.csv'
df2 = pd.read_csv(file3)

In [42]:
df2
import numpy as np

In [45]:
df2['Excess fare']= df2.groupby(['Embarked', 'Pclass'])[['Fare']].transform(lambda x: x-np.min(x))
df2

,PassengerId,Survived,Pclass,Name,Sex,Age,Ticket,Fare,Cabin,Embarked,Excess fare
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,A/5 21171,7.2500,NaN,S,7.2500
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,PC 17599,71.2833,C85,C,44.7333
2,3,1,3,"Heikkinen, Miss Laina",female,26.0,STON/O2. 3101282,7.9250,NaN,S,7.9250
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,113803,53.1000,C123,S,53.1000
4,5,0,3,"Allen, Mr. William Henry",male,35.0,373450,8.0500,NaN,S,8.0500
...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,211536,13.0000,NaN,S,13.0000
887,888,1,1,"Graham, Miss Margaret Edith",female,19.0,112053,30.0000,B42,S,30.0000
888,889,0,3,"Johnston, Miss Catherine Helen ""Carrie""",female,NaN,W./C. 6607,23.4500,NaN,S,23.4500
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,111369,30.0000,C148,C,3.4500


***
## Resampling and aggregation

We discussed how to handle time series data in pandas in the previous lecture. This basically comes down
to specifying an index which is a date or time stamp. Such and index allows us to easily perform operations such as computing leads, lags, and differences over time.

Another useful feature of the time series support in pandas is *resampling* which is used to group observations by time period and apply some aggregation function.
This can be accomplished using the 
[`resample()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.resample.html)
method which in its simplest form takes a string argument that describes how observations should be grouped
(`'YE'` for aggregation to years, `'QE'` for quarters, `'ME'` for months, `'W'` for weeks, etc.).

To illustrate, we load a data set that contains daily observations on the value of the NASDAQ stock market index at close:

In [19]:
# Path to NASDAQ data file
file = f'{DATA_PATH}/stockmarket/NASDAQ.csv'

# Read in NASDAQ data, set Date column as index
df = pd.read_csv(file, index_col='Date', parse_dates=True)

# Keep observations after 2024
df = df.loc['2024':]

# Print first few rows
df.head()

,NASDAQ
Date,
2024-01-02,14765.9
2024-01-03,14592.2
2024-01-04,14510.3
2024-01-05,14524.1
2024-01-08,14843.8


For example, if we want to aggregate this daily data to monthly frequency, we would use `resample('ME')`. This returns an object which is very similar to the one returned by `groupby()` we studied previously, and we can call various aggregation methods such as `mean()`:

In [20]:
# Resample to monthly frequency, aggregate to mean of daily observations 
# within each month
df.resample('ME').mean()

,NASDAQ
Date,
2024-01-31,15081.390476
2024-02-29,15808.935000
2024-03-31,16216.295000
2024-04-30,15950.868182
2024-05-31,16536.322727
2024-06-30,17495.900000
2024-07-31,17963.281818
2024-08-31,17268.263636
2024-09-30,17599.235000


Similarly, we can use `resample('W')` to resample to weekly frequency. Below,
we combine this with the aggregator 
[`last()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.last.html) 
to return the last observation of each week (weeks by default start on Sundays):

In [21]:
# Return last observation of each week, print first 10 rows
df.resample('W').last().head(10)

,NASDAQ
Date,
2024-01-07,14524.1
2024-01-14,14972.8
2024-01-21,15311.0
2024-01-28,15455.4
2024-02-04,15629.0
2024-02-11,15990.7
2024-02-18,15775.7
2024-02-25,15996.8
2024-03-03,16274.9


<div class="alert alert-info">
<h3> Your turn</h3>
<p>
Use the daily NASDAQ data for 2024 and compute the percentage change from the first to the last trading day within each month.
</p>
</div>

In [48]:
file3 = f'{DATA_PATH}/stockmarket/NASDAQ.csv'

# Read in NASDAQ data, set Date column as index
df3 = pd.read_csv(file3, index_col='Date', parse_dates=True)
df3

,NASDAQ
Date,
1971-02-05,100.0
1971-02-08,100.8
1971-02-09,100.8
1971-02-10,100.7
1971-02-11,101.4
...,...
2024-12-23,19764.9
2024-12-24,20031.1
2024-12-26,20020.4


In [56]:
df3 = df3.loc['2024':]
df3.resample('ME').agg(['first', 'last'])
df3['pct change'] = df3['NASDAQ'].pct_change()
df3

,NASDAQ,pct change
Date,,
2024-01-02,14765.9,NaN
2024-01-03,14592.2,-0.011764
2024-01-04,14510.3,-0.005613
2024-01-05,14524.1,0.000951
2024-01-08,14843.8,0.022012
...,...,...
2024-12-23,19764.9,0.009825
2024-12-24,20031.1,0.013468
2024-12-26,20020.4,-0.000534
